# Codec-Robust Adversarial Audio Generation with LLM Orchestration

This notebook implements a comprehensive pipeline for developing and evaluating codec-robust adversarial audio that reliably degrades ASR (Automatic Speech Recognition) performance after lossy compression.

## Methodology Overview

1. **Normalize audio** (target sample rate, loudness) and define clean ASR baselines
2. **Codec detection** and expose codec/bitrate candidates to LLM
3. **LLM-guided strategy generation** (Gemini 3) proposes perturbation strategies
4. **Executor** generates adversarial candidates with EoT (Expectation-over-Transformations)
5. **Metrics computation** (WER/CER, PESQ, STOI, SNR, LUFS)
6. **Feedback loop** returns structured summary to LLM for iteration
7. **Ablations** and statistical analysis
8. **Artifact saving** for reproducibility


In [ ]:
# Install required packages
!pip install -q google-generativeai openai-whisper librosa soundfile pesq pystoi numpy scipy matplotlib seaborn pandas scikit-learn ffmpeg-python


In [ ]:
import os
import json
import random
import subprocess
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import google.generativeai as genai
import whisper
from pesq import pesq
from pystoi import stoi
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


## Important: Set Your Gemini API Key

Before running the notebook, make sure to set your Gemini API key:

1. Get your API key from [Google AI Studio](https://makersuite.google.com/app/apikey)
2. Set it as an environment variable: `export GEMINI_API_KEY="your-key-here"`
3. Or update the `GEMINI_API_KEY` variable in the configuration cell below


di

In [ ]:
# Configuration
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "YOUR_API_KEY_HERE")  # Set your API key
DATASET_PATH = Path("/Users/kunal/Downloads/adversarial_dataset-A/Adversarial-Examples")
OUTPUT_DIR = Path("./agent_orchestration_outputs")
ARTIFACTS_DIR = OUTPUT_DIR / "artifacts"
RESULTS_DIR = OUTPUT_DIR / "results"

# Create output directories
OUTPUT_DIR.mkdir(exist_ok=True)
ARTIFACTS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Audio processing parameters
TARGET_SR = 16000
NORMALIZE_PEAK = 0.99
TARGET_LUFS = -23.0  # Broadcast standard

# Codec configuration
CODECS = {
    "mp3": {"codec": "libmp3lame", "bitrates": [64, 128, 192, 256]},
    "aac": {"codec": "aac", "bitrates": [64, 128, 192, 256]},
    "opus": {"codec": "libopus", "bitrates": [32, 64, 96, 128]},
    "amr-wb": {"codec": "libopencore_amrwb", "bitrates": [6.6, 8.85, 12.65, 14.25, 15.85, 18.25, 19.85, 23.05, 23.85]},
    "g711": {"codec": "pcm_mulaw", "bitrates": [64]}  # Fixed bitrate
}

# ASR model
ASR_MODEL_NAME = "base"  # Options: tiny, base, small, medium, large

# EoT (Expectation over Transformations) parameters
EOT_NUM_SAMPLES = 10  # Number of codec transformations per optimization step
EOT_CHAIN_LENGTH = 3  # Maximum transcoding chain length

# Perturbation constraints
MAX_LINF = 0.01  # Maximum L∞ norm
MAX_L2 = 0.1    # Maximum L2 norm
MIN_PESQ = 3.0  # Minimum PESQ score
MIN_STOI = 0.7  # Minimum STOI score
MIN_SNR = 10.0  # Minimum SNR in dB
TARGET_SNR = 20.0  # Target SNR in dB

# LLM parameters
MAX_ITERATIONS = 5  # Maximum feedback loop iterations
STRATEGY_TOP_K = 3  # Top-k strategies to return to LLM

print("Configuration loaded successfully!")


In [ ]:
class AudioNormalizer:
    """Normalize audio to target sample rate and loudness."""
    
    def __init__(self, target_sr: int = TARGET_SR, target_lufs: float = TARGET_LUFS, peak: float = NORMALIZE_PEAK):
        self.target_sr = target_sr
        self.target_lufs = target_lufs
        self.peak = peak
    
    def normalize(self, audio_path: Path) -> Tuple[np.ndarray, int]:
        """Load and normalize audio file."""
        # Load audio
        audio, sr = librosa.load(str(audio_path), sr=self.target_sr, mono=True)
        
        # Peak normalization
        peak_val = np.max(np.abs(audio))
        if peak_val > 0:
            audio = audio * (self.peak / peak_val)
        
        # LUFS normalization (simplified - using RMS approximation)
        # For production, use pyloudnorm or similar
        rms = np.sqrt(np.mean(audio**2))
        target_rms = 10 ** (self.target_lufs / 20) * 0.1  # Approximate conversion
        if rms > 0:
            audio = audio * (target_rms / rms)
        
        # Clip to prevent overflow
        audio = np.clip(audio, -1.0, 1.0)
        
        return audio.astype(np.float32), self.target_sr
    
    def save_normalized(self, audio: np.ndarray, output_path: Path, sr: int = TARGET_SR):
        """Save normalized audio to file."""
        sf.write(str(output_path), audio, sr)


class ASRBaseline:
    """Whisper-based ASR baseline for transcription."""
    
    def __init__(self, model_name: str = ASR_MODEL_NAME):
        print(f"Loading Whisper model: {model_name}")
        self.model = whisper.load_model(model_name)
        self.model_name = model_name
    
    def transcribe(self, audio: np.ndarray, sr: int = TARGET_SR) -> str:
        """Transcribe audio to text."""
        # Whisper expects float32 audio in range [-1, 1]
        if audio.dtype != np.float32:
            audio = audio.astype(np.float32)
        
        # Resample if needed (Whisper expects 16kHz)
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        
        result = self.model.transcribe(audio, language="en", fp16=False)
        return result["text"].strip()
    
    def compute_wer(self, reference: str, hypothesis: str) -> float:
        """Compute Word Error Rate (WER)."""
        ref_words = reference.lower().split()
        hyp_words = hypothesis.lower().split()
        
        if len(ref_words) == 0:
            return 1.0 if len(hyp_words) > 0 else 0.0
        
        # Dynamic programming for edit distance
        d = np.zeros((len(ref_words) + 1, len(hyp_words) + 1))
        for i in range(len(ref_words) + 1):
            d[i, 0] = i
        for j in range(len(hyp_words) + 1):
            d[0, j] = j
        
        for i in range(1, len(ref_words) + 1):
            for j in range(1, len(hyp_words) + 1):
                if ref_words[i-1] == hyp_words[j-1]:
                    d[i, j] = d[i-1, j-1]
                else:
                    d[i, j] = min(
                        d[i-1, j] + 1,      # deletion
                        d[i, j-1] + 1,      # insertion
                        d[i-1, j-1] + 1     # substitution
                    )
        
        return d[len(ref_words), len(hyp_words)] / len(ref_words)
    
    def compute_cer(self, reference: str, hypothesis: str) -> float:
        """Compute Character Error Rate (CER)."""
        ref_chars = list(reference.lower().replace(" ", ""))
        hyp_chars = list(hypothesis.lower().replace(" ", ""))
        
        if len(ref_chars) == 0:
            return 1.0 if len(hyp_chars) > 0 else 0.0
        
        # Character-level edit distance
        d = np.zeros((len(ref_chars) + 1, len(hyp_chars) + 1))
        for i in range(len(ref_chars) + 1):
            d[i, 0] = i
        for j in range(len(hyp_chars) + 1):
            d[0, j] = j
        
        for i in range(1, len(ref_chars) + 1):
            for j in range(1, len(hyp_chars) + 1):
                if ref_chars[i-1] == hyp_chars[j-1]:
                    d[i, j] = d[i-1, j-1]
                else:
                    d[i, j] = min(
                        d[i-1, j] + 1,
                        d[i, j-1] + 1,
                        d[i-1, j-1] + 1
                    )
        
        return d[len(ref_chars), len(hyp_chars)] / len(ref_chars)


# Initialize components
normalizer = AudioNormalizer()
asr_baseline = ASRBaseline()

print("Audio normalizer and ASR baseline initialized!")


In [ ]:
class CodecDetector:
    """Detect codec information from audio files."""
    
    def detect(self, audio_path: Path) -> Dict[str, Any]:
        """Detect codec using ffprobe."""
        try:
            cmd = [
                "ffprobe", "-v", "quiet", "-print_format", "json", "-show_format",
                "-show_streams", str(audio_path)
            ]
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            data = json.loads(result.stdout)
            
            # Extract codec info
            stream = data.get("streams", [{}])[0]
            format_info = data.get("format", {})
            
            codec_name = stream.get("codec_name", "unknown")
            bitrate = int(format_info.get("bit_rate", 0)) // 1000  # Convert to kbps
            sample_rate = int(stream.get("sample_rate", TARGET_SR))
            channels = int(stream.get("channels", 1))
            container = format_info.get("format_name", "").split(",")[0]
            
            return {
                "codec_name": codec_name,
                "bitrate_kbps": bitrate,
                "sample_rate": sample_rate,
                "channels": channels,
                "container": container,
                "detected": True
            }
        except Exception as e:
            # Fallback to heuristic detection
            ext = audio_path.suffix.lower()
            mapping = {
                ".wav": {"codec_name": "pcm", "bitrate_kbps": 1411},
                ".mp3": {"codec_name": "mp3", "bitrate_kbps": 192},
                ".m4a": {"codec_name": "aac", "bitrate_kbps": 256},
                ".flac": {"codec_name": "flac", "bitrate_kbps": 1000},
                ".opus": {"codec_name": "opus", "bitrate_kbps": 96}
            }
            default = mapping.get(ext, {"codec_name": "unknown", "bitrate_kbps": 128})
            return {
                **default,
                "sample_rate": TARGET_SR,
                "channels": 1,
                "container": ext.lstrip("."),
                "detected": False,
                "error": str(e)
            }


class CodecStack:
    """Codec stack for transcoding audio."""
    
    # Map codec names to proper file extensions
    CODEC_EXTENSIONS = {
        "mp3": ".mp3",
        "aac": ".m4a",  # AAC typically uses .m4a container
        "opus": ".opus",
        "amr-wb": ".amr",
        "g711": ".ulaw"
    }
    
    def __init__(self, codecs: Dict[str, Dict] = CODECS):
        self.codecs = codecs
    
    def _get_output_path(self, output_path: Path, codec_name: str) -> Path:
        """Ensure output path has proper extension for codec."""
        ext = self.CODEC_EXTENSIONS.get(codec_name, ".tmp")
        if output_path.suffix != ext:
            # Replace extension
            return output_path.with_suffix(ext)
        return output_path
    
    def encode(self, audio_path: Path, codec_name: str, bitrate: Any, output_path: Path) -> bool:
        """Encode audio using specified codec and bitrate."""
        if codec_name not in self.codecs:
            raise ValueError(f"Unsupported codec: {codec_name}")
        
        codec_info = self.codecs[codec_name]
        codec = codec_info["codec"]
        
        # Ensure proper file extension
        output_path = self._get_output_path(output_path, codec_name)
        
        try:
            # Build ffmpeg command
            cmd = [
                "ffmpeg", "-y", "-i", str(audio_path),
                "-acodec", codec,
                "-ar", str(TARGET_SR),
                "-ac", "1"  # Mono
            ]
            
            # Add bitrate and format-specific options
            if codec_name == "g711":
                # G.711 is fixed bitrate
                cmd.extend(["-f", "mulaw"])
            elif codec_name == "amr-wb":
                cmd.extend(["-b:a", f"{bitrate}k"])
            elif codec_name == "aac":
                # AAC needs explicit format
                cmd.extend(["-b:a", f"{bitrate}k", "-f", "ipod"])
            else:
                cmd.extend(["-b:a", f"{bitrate}k"])
            
            cmd.append(str(output_path))
            
            # Run encoding
            result = subprocess.run(
                cmd, capture_output=True, text=True, check=True
            )
            return True
        except subprocess.CalledProcessError as e:
            print(f"Encoding failed: {e.stderr}")
            return False
    
    def decode(self, encoded_path: Path, output_path: Path) -> bool:
        """Decode encoded audio back to WAV."""
        try:
            cmd = [
                "ffmpeg", "-y", "-i", str(encoded_path),
                "-acodec", "pcm_s16le",
                "-ar", str(TARGET_SR),
                "-ac", "1",
                str(output_path)
            ]
            subprocess.run(cmd, capture_output=True, text=True, check=True)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Decoding failed: {e.stderr}")
            return False
    
    def apply_codec_chain(self, audio_path: Path, chain: List[Tuple[str, Any]], 
                         output_dir: Path) -> Optional[Path]:
        """Apply a chain of codec transformations."""
        current_path = audio_path
        temp_dir = output_dir / "temp_codec"
        temp_dir.mkdir(exist_ok=True)
        
        final_output = None
        for i, (codec_name, bitrate) in enumerate(chain):
            temp_output = temp_dir / f"chain_{i}_{codec_name}_{bitrate}.tmp"
            
            # Encode (this will automatically fix the extension)
            if not self.encode(current_path, codec_name, bitrate, temp_output):
                return None
            
            # Get the actual output path with correct extension
            actual_output = self._get_output_path(temp_output, codec_name)
            final_output = actual_output
            
            # Decode for next step
            if i < len(chain) - 1:
                temp_decoded = temp_dir / f"chain_{i}_decoded.wav"
                if not self.decode(actual_output, temp_decoded):
                    return None
                current_path = temp_decoded
        
        return final_output if len(chain) == 1 else current_path


# Initialize codec components
codec_detector = CodecDetector()
codec_stack = CodecStack()

print("Codec detector and stack initialized!")


In [ ]:
# Initialize Gemini
genai.configure(api_key=GEMINI_API_KEY)

@dataclass
class PerturbationStrategy:
    """Structured perturbation strategy from LLM."""
    name: str
    family: str  # e.g., "narrowband_spectral_noise", "phase_only", "micro_time_warp", "spread_spectrum"
    optimizer: str  # e.g., "CMA-ES", "gradient", "black_box"
    constraints: Dict[str, float]
    eot_schedule: Dict[str, Any]
    code_snippet: str
    parameters: Dict[str, Any] = field(default_factory=dict)
    description: str = ""


class LLMOrchestrator:
    """Gemini 3-based LLM orchestrator for strategy generation."""
    
    def __init__(self, model_name: str = None):
        # Try Gemini 3 models in order of preference
        gemini_models = [
            "gemini-2.0-flash-exp",  # Latest experimental
            "gemini-1.5-pro",        # Gemini 1.5 Pro
            "gemini-1.5-flash",      # Gemini 1.5 Flash
            "gemini-pro"             # Fallback
        ]
        
        if model_name:
            gemini_models.insert(0, model_name)
        
        self.model = None
        self.model_name = None
        
        for model in gemini_models:
            try:
                self.model = genai.GenerativeModel(model)
                self.model_name = model
                print(f"Successfully loaded Gemini model: {model}")
                break
            except Exception as e:
                print(f"Could not load {model}: {e}")
                continue
        
        if self.model is None:
            raise RuntimeError("Failed to load any Gemini model. Please check your API key and model availability.")
    
    def generate_strategy(
        self,
        codec_info: Dict[str, Any],
        available_codecs: Dict[str, Dict],
        previous_feedback: Optional[str] = None,
        iteration: int = 1
    ) -> PerturbationStrategy:
        """Generate perturbation strategy based on codec information."""
        
        # Build prompt
        prompt = self._build_strategy_prompt(
            codec_info, available_codecs, previous_feedback, iteration
        )
        
        try:
            response = self.model.generate_content(prompt)
            strategy_text = response.text
            
            # Parse strategy from response
            strategy = self._parse_strategy(strategy_text, codec_info)
            return strategy
        except Exception as e:
            print(f"LLM generation failed: {e}")
            # Return default strategy
            return self._default_strategy(codec_info)
    
    def _build_strategy_prompt(
        self,
        codec_info: Dict[str, Any],
        available_codecs: Dict[str, Dict],
        previous_feedback: Optional[str],
        iteration: int
    ) -> str:
        """Build prompt for strategy generation."""
        
        codec_list = ", ".join([f"{k} (bitrates: {v['bitrates']})" 
                                for k, v in available_codecs.items()])
        
        feedback_section = ""
        if previous_feedback:
            feedback_section = f"""
Previous iteration feedback:
{previous_feedback}

Based on this feedback, revise your strategy.
"""
        
        prompt = f"""You are an expert in adversarial audio generation for ASR systems. Your task is to design perturbation strategies that survive lossy codec compression.

Current codec context:
- Detected codec: {codec_info.get('codec_name', 'unknown')}
- Bitrate: {codec_info.get('bitrate_kbps', 'unknown')} kbps
- Sample rate: {codec_info.get('sample_rate', TARGET_SR)} Hz

Available codecs for EoT (Expectation over Transformations):
{codec_list}

Constraints:
- Maximum L∞ norm: {MAX_LINF}
- Maximum L2 norm: {MAX_L2}
- Minimum PESQ: {MIN_PESQ}
- Minimum STOI: {MIN_STOI}
- Minimum SNR: {MIN_SNR} dB
- Target SNR: {TARGET_SNR} dB

{feedback_section}

Generate a perturbation strategy in the following JSON format:
{{
    "name": "strategy_name",
    "family": "narrowband_spectral_noise|phase_only|micro_time_warp|spread_spectrum|hybrid",
    "optimizer": "CMA-ES|gradient|black_box",
    "constraints": {{
        "max_linf": {MAX_LINF},
        "max_l2": {MAX_L2},
        "min_pesq": {MIN_PESQ},
        "min_stoi": {MIN_STOI},
        "min_snr": {MIN_SNR},
        "target_snr": {TARGET_SNR}
    }},
    "eot_schedule": {{
        "num_samples": {EOT_NUM_SAMPLES},
        "codec_mix": ["mp3", "aac", "opus"],
        "bitrate_range": [64, 256],
        "chain_length": {EOT_CHAIN_LENGTH},
        "chain_probability": 0.3
    }},
    "parameters": {{
        "frequency_bands": [3000, 4000],
        "noise_level": 0.005,
        "time_warp_factor": 0.01
    }},
    "code_snippet": "Python code implementing the perturbation",
    "description": "Detailed description of the strategy"
}}

Focus on strategies that:
1. Exploit codec-specific vulnerabilities (e.g., MP3's frequency masking, AAC's temporal windows)
2. Use frequency-domain perturbations that survive quantization
3. Apply phase-only modifications that are less perceptible
4. Leverage EoT to ensure robustness across codec chains

Return ONLY the JSON, no additional text."""
        
        return prompt
    
    def _parse_strategy(self, response_text: str, codec_info: Dict[str, Any]) -> PerturbationStrategy:
        """Parse strategy from LLM response."""
        try:
            # Extract JSON from response
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                strategy_dict = json.loads(json_match.group())
            else:
                raise ValueError("No JSON found in response")
            
            return PerturbationStrategy(
                name=strategy_dict.get("name", "default_strategy"),
                family=strategy_dict.get("family", "narrowband_spectral_noise"),
                optimizer=strategy_dict.get("optimizer", "CMA-ES"),
                constraints=strategy_dict.get("constraints", {}),
                eot_schedule=strategy_dict.get("eot_schedule", {}),
                code_snippet=strategy_dict.get("code_snippet", ""),
                parameters=strategy_dict.get("parameters", {}),
                description=strategy_dict.get("description", "")
            )
        except Exception as e:
            print(f"Failed to parse strategy: {e}")
            return self._default_strategy(codec_info)
    
    def _default_strategy(self, codec_info: Dict[str, Any]) -> PerturbationStrategy:
        """Return default strategy if LLM fails."""
        return PerturbationStrategy(
            name="default_narrowband_noise",
            family="narrowband_spectral_noise",
            optimizer="CMA-ES",
            constraints={
                "max_linf": MAX_LINF,
                "max_l2": MAX_L2,
                "min_pesq": MIN_PESQ,
                "min_stoi": MIN_STOI,
                "min_snr": MIN_SNR,
                "target_snr": TARGET_SNR
            },
            eot_schedule={
                "num_samples": EOT_NUM_SAMPLES,
                "codec_mix": ["mp3", "aac"],
                "bitrate_range": [64, 192],
                "chain_length": 2,
                "chain_probability": 0.2
            },
            code_snippet="""
def apply_perturbation(audio, sr=16000):
    # Narrowband noise injection
    noise = np.random.randn(len(audio)) * 0.005
    # Filter to 3-4 kHz band
    from scipy import signal
    b, a = signal.butter(4, [3000/(sr/2), 4000/(sr/2)], btype='band')
    noise = signal.filtfilt(b, a, noise)
    return audio + noise
            """,
            parameters={"frequency_bands": [3000, 4000], "noise_level": 0.005},
            description="Default narrowband spectral noise injection"
        )
    
    def generate_feedback_summary(
        self,
        results: List[Dict[str, Any]],
        top_k: int = STRATEGY_TOP_K
    ) -> str:
        """Generate feedback summary for LLM based on results."""
        if not results:
            return "No results available yet."
        
        # Sort by WER increase (best attacks first)
        sorted_results = sorted(
            results,
            key=lambda x: x.get("wer_delta", 0),
            reverse=True
        )
        
        top_results = sorted_results[:top_k]
        
        summary = f"Top {len(top_results)} strategies:\n\n"
        for i, result in enumerate(top_results, 1):
            summary += f"{i}. {result.get('strategy_name', 'unknown')}:\n"
            summary += f"   - WER delta: {result.get('wer_delta', 0):.3f}\n"
            summary += f"   - CER delta: {result.get('cer_delta', 0):.3f}\n"
            summary += f"   - PESQ: {result.get('pesq', 0):.2f}\n"
            summary += f"   - STOI: {result.get('stoi', 0):.3f}\n"
            summary += f"   - SNR: {result.get('snr', 0):.2f} dB\n"
            summary += f"   - Codec: {result.get('codec', 'unknown')}\n\n"
        
        # Failure modes
        failures = [r for r in results if r.get("wer_delta", 0) < 0.1]
        if failures:
            summary += f"\nFailure modes ({len(failures)} strategies):\n"
            summary += "- Low WER increase despite perturbation\n"
            summary += "- Constraint violations (PESQ/STOI too low)\n"
            summary += "- Codec-specific robustness issues\n"
        
        return summary


# Initialize LLM orchestrator
llm_orchestrator = LLMOrchestrator()

print("LLM orchestrator initialized!")


In [ ]:
class PerturbationExecutor:
    """Execute perturbation strategies on audio."""
    
    def __init__(self, codec_stack: CodecStack, normalizer: AudioNormalizer):
        self.codec_stack = codec_stack
        self.normalizer = normalizer
    
    def apply_perturbation(
        self,
        audio: np.ndarray,
        strategy: PerturbationStrategy,
        sr: int = TARGET_SR
    ) -> np.ndarray:
        """Apply perturbation based on strategy."""
        # Execute the code snippet from strategy
        try:
            # Create execution context
            exec_globals = {
                "np": np,
                "librosa": librosa,
                "audio": audio.copy(),
                "sr": sr,
                "strategy": strategy
            }
            
            # Execute code snippet
            exec(strategy.code_snippet, exec_globals)
            
            # Get perturbed audio (assuming function returns it)
            if "perturbed_audio" in exec_globals:
                perturbed = exec_globals["perturbed_audio"]
            elif "result" in exec_globals:
                perturbed = exec_globals["result"]
            else:
                # Fallback: apply default perturbation
                perturbed = self._apply_default_perturbation(audio, strategy, sr)
            
            # Ensure constraints
            perturbed = self._enforce_constraints(audio, perturbed, strategy)
            
            return perturbed.astype(np.float32)
        except Exception as e:
            print(f"Perturbation execution failed: {e}")
            return self._apply_default_perturbation(audio, strategy, sr)
    
    def _apply_default_perturbation(
        self,
        audio: np.ndarray,
        strategy: PerturbationStrategy,
        sr: int
    ) -> np.ndarray:
        """Apply default perturbation based on family."""
        family = strategy.family.lower()
        params = strategy.parameters
        
        if "narrowband" in family or "spectral" in family:
            # Narrowband spectral noise
            noise_level = params.get("noise_level", 0.005)
            freq_bands = params.get("frequency_bands", [3000, 4000])
            
            from scipy import signal
            noise = np.random.randn(len(audio)) * noise_level
            b, a = signal.butter(4, [freq_bands[0]/(sr/2), freq_bands[1]/(sr/2)], btype='band')
            noise = signal.filtfilt(b, a, noise)
            return audio + noise
        
        elif "phase" in family:
            # Phase-only modification
            fft = np.fft.fft(audio)
            magnitude = np.abs(fft)
            phase = np.angle(fft)
            phase_shift = params.get("phase_shift", 0.01) * np.random.randn(len(phase))
            new_fft = magnitude * np.exp(1j * (phase + phase_shift))
            return np.real(np.fft.ifft(new_fft))
        
        elif "time_warp" in family:
            # Micro time warping
            from scipy.interpolate import interp1d
            warp_factor = params.get("time_warp_factor", 0.01)
            n = len(audio)
            indices = np.arange(n) + warp_factor * np.sin(2 * np.pi * np.arange(n) / (n/10))
            indices = np.clip(indices, 0, n-1)
            f = interp1d(np.arange(n), audio, kind='linear', fill_value='extrapolate')
            return f(indices)
        
        elif "spread_spectrum" in family:
            # Spread spectrum pattern
            noise_level = params.get("noise_level", 0.003)
            noise = np.random.randn(len(audio)) * noise_level
            # Modulate with chirp
            t = np.arange(len(audio)) / sr
            chirp = np.sin(2 * np.pi * (1000 + 2000 * t) * t)
            noise = noise * (1 + 0.1 * chirp)
            return audio + noise
        
        else:
            # Default: additive Gaussian noise
            noise_level = params.get("noise_level", 0.005)
            noise = np.random.randn(len(audio)) * noise_level
            return audio + noise
    
    def _enforce_constraints(
        self,
        original: np.ndarray,
        perturbed: np.ndarray,
        strategy: PerturbationStrategy
    ) -> np.ndarray:
        """Enforce perturbation constraints."""
        constraints = strategy.constraints
        perturbation = perturbed - original
        
        # L∞ constraint
        max_linf = constraints.get("max_linf", MAX_LINF)
        if np.max(np.abs(perturbation)) > max_linf:
            perturbation = perturbation * (max_linf / np.max(np.abs(perturbation)))
        
        # L2 constraint
        max_l2 = constraints.get("max_l2", MAX_L2)
        l2_norm = np.linalg.norm(perturbation)
        if l2_norm > max_l2:
            perturbation = perturbation * (max_l2 / l2_norm)
        
        perturbed = original + perturbation
        return np.clip(perturbed, -1.0, 1.0)
    
    def apply_eot(
        self,
        audio: np.ndarray,
        strategy: PerturbationStrategy,
        output_dir: Path,
        sr: int = TARGET_SR
    ) -> List[Tuple[np.ndarray, Dict[str, Any]]]:
        """Apply Expectation over Transformations."""
        eot_schedule = strategy.eot_schedule
        num_samples = eot_schedule.get("num_samples", EOT_NUM_SAMPLES)
        codec_mix = eot_schedule.get("codec_mix", ["mp3", "aac"])
        bitrate_range = eot_schedule.get("bitrate_range", [64, 192])
        chain_length = eot_schedule.get("chain_length", EOT_CHAIN_LENGTH)
        chain_prob = eot_schedule.get("chain_probability", 0.3)
        
        results = []
        temp_dir = output_dir / "eot_temp"
        temp_dir.mkdir(exist_ok=True)
        
        # Save original perturbed audio
        temp_original = temp_dir / "perturbed_original.wav"
        sf.write(str(temp_original), audio, sr)
        
        for i in range(num_samples):
            # Decide on chain vs single codec
            use_chain = np.random.random() < chain_prob and chain_length > 1
            
            if use_chain:
                # Generate codec chain
                chain = []
                for _ in range(np.random.randint(2, chain_length + 1)):
                    codec = np.random.choice(codec_mix)
                    bitrate = np.random.choice(CODECS[codec]["bitrates"])
                    chain.append((codec, bitrate))
                
                # Apply chain
                transformed_path = self.codec_stack.apply_codec_chain(
                    temp_original, chain, temp_dir
                )
            else:
                # Single codec transformation
                codec = np.random.choice(codec_mix)
                bitrate = np.random.choice(CODECS[codec]["bitrates"])
                transformed_path = temp_dir / f"eot_{i}_{codec}_{bitrate}.tmp"
                if self.codec_stack.encode(temp_original, codec, bitrate, transformed_path):
                    # Get the actual path with correct extension
                    transformed_path = self.codec_stack._get_output_path(transformed_path, codec)
                else:
                    transformed_path = None
            
            if transformed_path and transformed_path.exists():
                # Decode back to audio
                decoded_path = temp_dir / f"eot_{i}_decoded.wav"
                if self.codec_stack.decode(transformed_path, decoded_path):
                    transformed_audio, _ = self.normalizer.normalize(decoded_path)
                    
                    metadata = {
                        "sample_idx": i,
                        "codec": codec if not use_chain else "chain",
                        "bitrate": bitrate if not use_chain else None,
                        "chain": chain if use_chain else None
                    }
                    results.append((transformed_audio, metadata))
        
        return results


# Initialize executor
perturbation_executor = PerturbationExecutor(codec_stack, normalizer)

print("Perturbation executor initialized!")


In [ ]:
class MetricsComputer:
    """Compute all evaluation metrics."""
    
    def __init__(self, asr_baseline: ASRBaseline):
        self.asr = asr_baseline
    
    def compute_all_metrics(
        self,
        original_audio: np.ndarray,
        perturbed_audio: np.ndarray,
        original_transcript: str,
        sr: int = TARGET_SR
    ) -> Dict[str, float]:
        """Compute all metrics for a perturbed audio sample."""
        metrics = {}
        
        # ASR metrics
        perturbed_transcript = self.asr.transcribe(perturbed_audio, sr)
        metrics["wer"] = self.asr.compute_wer(original_transcript, perturbed_transcript)
        metrics["cer"] = self.asr.compute_cer(original_transcript, perturbed_transcript)
        metrics["perturbed_transcript"] = perturbed_transcript
        
        # Perceptual quality metrics
        min_len = min(len(original_audio), len(perturbed_audio))
        orig_trimmed = original_audio[:min_len]
        pert_trimmed = perturbed_audio[:min_len]
        
        try:
            metrics["pesq"] = pesq(sr, orig_trimmed, pert_trimmed, 'wb')
        except:
            metrics["pesq"] = 0.0
        
        try:
            metrics["stoi"] = stoi(orig_trimmed, pert_trimmed, sr, extended=False)
        except:
            metrics["stoi"] = 0.0
        
        # Signal metrics
        perturbation = pert_trimmed - orig_trimmed
        signal_power = np.mean(orig_trimmed ** 2)
        noise_power = np.mean(perturbation ** 2)
        if noise_power > 0:
            metrics["snr"] = 10 * np.log10(signal_power / noise_power)
        else:
            metrics["snr"] = float('inf')
        
        # LUFS (simplified RMS-based approximation)
        rms_pert = np.sqrt(np.mean(pert_trimmed ** 2))
        metrics["lufs"] = 20 * np.log10(rms_pert / 0.1) if rms_pert > 0 else -np.inf
        
        # Norms
        metrics["l2_norm"] = np.linalg.norm(perturbation)
        metrics["linf_norm"] = np.max(np.abs(perturbation))
        
        return metrics
    
    def compute_baseline_metrics(
        self,
        original_audio: np.ndarray,
        original_transcript: str,
        sr: int = TARGET_SR
    ) -> Dict[str, float]:
        """Compute baseline metrics for original audio."""
        # Verify transcription
        verified_transcript = self.asr.transcribe(original_audio, sr)
        wer = self.asr.compute_wer(original_transcript, verified_transcript)
        cer = self.asr.compute_cer(original_transcript, verified_transcript)
        
        return {
            "wer": wer,
            "cer": cer,
            "transcript": verified_transcript,
            "snr": float('inf'),  # No noise in original
            "pesq": 5.0,  # Perfect quality
            "stoi": 1.0   # Perfect intelligibility
        }
    
    def compute_delta_metrics(
        self,
        baseline: Dict[str, float],
        perturbed: Dict[str, float]
    ) -> Dict[str, float]:
        """Compute delta metrics (perturbed - baseline)."""
        return {
            "wer_delta": perturbed["wer"] - baseline["wer"],
            "cer_delta": perturbed["cer"] - baseline["cer"],
            "pesq_delta": perturbed["pesq"] - baseline["pesq"],
            "stoi_delta": perturbed["stoi"] - baseline["stoi"],
            "snr_delta": perturbed["snr"] - baseline["snr"] if baseline["snr"] != float('inf') else -perturbed["snr"]
        }


# Initialize metrics computer
metrics_computer = MetricsComputer(asr_baseline)

print("Metrics computer initialized!")


## Step 6-8: Main Orchestration Pipeline


In [ ]:
@dataclass
class ExperimentResult:
    """Results from a single experiment run."""
    audio_file: str
    strategy_name: str
    iteration: int
    baseline_metrics: Dict[str, float]
    perturbed_metrics: Dict[str, float]
    eot_results: List[Dict[str, Any]]
    delta_metrics: Dict[str, float]
    codec_info: Dict[str, Any]
    strategy: Dict[str, Any]
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())


class AgentOrchestrator:
    """Main orchestrator for the entire pipeline."""
    
    def __init__(
        self,
        normalizer: AudioNormalizer,
        asr_baseline: ASRBaseline,
        codec_detector: CodecDetector,
        codec_stack: CodecStack,
        llm_orchestrator: LLMOrchestrator,
        perturbation_executor: PerturbationExecutor,
        metrics_computer: MetricsComputer
    ):
        self.normalizer = normalizer
        self.asr_baseline = asr_baseline
        self.codec_detector = codec_detector
        self.codec_stack = codec_stack
        self.llm_orchestrator = llm_orchestrator
        self.perturbation_executor = perturbation_executor
        self.metrics_computer = metrics_computer
    
    def run_experiment(
        self,
        audio_path: Path,
        reference_transcript: Optional[str] = None,
        max_iterations: int = MAX_ITERATIONS
    ) -> List[ExperimentResult]:
        """Run full experiment pipeline."""
        print(f"\n{'='*80}")
        print(f"Starting experiment for: {audio_path.name}")
        print(f"{'='*80}\n")
        
        # Step 1: Normalize audio
        print("Step 1: Normalizing audio...")
        original_audio, sr = self.normalizer.normalize(audio_path)
        print(f"  Loaded audio: {len(original_audio)/sr:.2f}s, {sr} Hz")
        
        # Get or generate reference transcript
        if reference_transcript is None:
            print("  Generating reference transcript...")
            reference_transcript = self.asr_baseline.transcribe(original_audio, sr)
        print(f"  Reference: '{reference_transcript}'")
        
        # Compute baseline metrics
        print("\nStep 2: Computing baseline metrics...")
        baseline_metrics = self.metrics_computer.compute_baseline_metrics(
            original_audio, reference_transcript, sr
        )
        print(f"  Baseline WER: {baseline_metrics['wer']:.3f}")
        print(f"  Baseline CER: {baseline_metrics['cer']:.3f}")
        
        # Step 2: Detect codec
        print("\nStep 3: Detecting codec...")
        codec_info = self.codec_detector.detect(audio_path)
        print(f"  Codec: {codec_info['codec_name']}")
        print(f"  Bitrate: {codec_info['bitrate_kbps']} kbps")
        
        # Main feedback loop
        all_results = []
        previous_feedback = None
        
        for iteration in range(1, max_iterations + 1):
            print(f"\n{'='*80}")
            print(f"Iteration {iteration}/{max_iterations}")
            print(f"{'='*80}\n")
            
            # Show feedback from previous iteration (if any)
            if previous_feedback:
                print("📋 Feedback from previous iteration:")
                print("-" * 80)
                print(previous_feedback)
                print("-" * 80)
                print()
            
            # Step 3: Generate strategy from LLM
            print("Step 4: Generating perturbation strategy from LLM...")
            strategy = self.llm_orchestrator.generate_strategy(
                codec_info, CODECS, previous_feedback, iteration
            )
            print(f"  Strategy: {strategy.name}")
            print(f"  Family: {strategy.family}")
            print(f"  Optimizer: {strategy.optimizer}")
            
            # Step 4: Apply perturbation
            print("\nStep 5: Applying perturbation...")
            perturbed_audio = self.perturbation_executor.apply_perturbation(
                original_audio, strategy, sr
            )
            print(f"  Perturbation applied: L∞={np.max(np.abs(perturbed_audio - original_audio)):.6f}")
            
            # Step 4b: Apply EoT
            print("\nStep 6: Applying Expectation over Transformations...")
            eot_outputs = self.perturbation_executor.apply_eot(
                perturbed_audio, strategy, ARTIFACTS_DIR, sr
            )
            print(f"  Generated {len(eot_outputs)} EoT samples")
            
            # Step 5: Compute metrics for perturbed audio
            print("\nStep 7: Computing metrics...")
            perturbed_metrics = self.metrics_computer.compute_all_metrics(
                original_audio, perturbed_audio, reference_transcript, sr
            )
            delta_metrics = self.metrics_computer.compute_delta_metrics(
                baseline_metrics, perturbed_metrics
            )
            
            print(f"  WER: {perturbed_metrics['wer']:.3f} (Δ: {delta_metrics['wer_delta']:+.3f})")
            print(f"  CER: {perturbed_metrics['cer']:.3f} (Δ: {delta_metrics['cer_delta']:+.3f})")
            print(f"  PESQ: {perturbed_metrics['pesq']:.2f}")
            print(f"  STOI: {perturbed_metrics['stoi']:.3f}")
            print(f"  SNR: {perturbed_metrics['snr']:.2f} dB")
            
            # Compute metrics for EoT samples
            eot_results = []
            for eot_audio, eot_metadata in eot_outputs:
                eot_metrics = self.metrics_computer.compute_all_metrics(
                    original_audio, eot_audio, reference_transcript, sr
                )
                eot_delta = self.metrics_computer.compute_delta_metrics(
                    baseline_metrics, eot_metrics
                )
                eot_results.append({
                    **eot_metadata,
                    **eot_metrics,
                    **eot_delta
                })
            
            # Create result
            result = ExperimentResult(
                audio_file=str(audio_path),
                strategy_name=strategy.name,
                iteration=iteration,
                baseline_metrics=baseline_metrics,
                perturbed_metrics=perturbed_metrics,
                eot_results=eot_results,
                delta_metrics=delta_metrics,
                codec_info=codec_info,
                strategy=asdict(strategy)
            )
            all_results.append(result)
            
            # Step 6: Generate feedback for next iteration
            print("\nStep 8: Generating feedback summary...")
            feedback_data = [{
                "strategy_name": strategy.name,
                "wer_delta": delta_metrics["wer_delta"],
                "cer_delta": delta_metrics["cer_delta"],
                "pesq": perturbed_metrics["pesq"],
                "stoi": perturbed_metrics["stoi"],
                "snr": perturbed_metrics["snr"],
                "codec": codec_info["codec_name"]
            }]
            
            previous_feedback = self.llm_orchestrator.generate_feedback_summary(
                feedback_data, top_k=1
            )
            print(f"\n📊 Feedback summary for next iteration:")
            print("=" * 80)
            print(previous_feedback)
            print("=" * 80)
            
            # Check if we've achieved good results
            if delta_metrics["wer_delta"] > 0.3 and perturbed_metrics["pesq"] >= MIN_PESQ:
                print(f"\n✓ Good results achieved! WER delta: {delta_metrics['wer_delta']:.3f}")
                break
        
        return all_results
    
    def save_results(self, results: List[ExperimentResult], output_path: Path):
        """Save experiment results to JSON."""
        results_dict = [asdict(r) for r in results]
        with open(output_path, 'w') as f:
            json.dump(results_dict, f, indent=2, default=str)
        print(f"\nResults saved to: {output_path}")
    
    def save_artifacts(
        self,
        original_audio: np.ndarray,
        perturbed_audio: np.ndarray,
        audio_name: str,
        strategy_name: str,
        iteration: int,
        sr: int = TARGET_SR
    ):
        """Save audio artifacts."""
        artifact_dir = ARTIFACTS_DIR / audio_name / f"iter_{iteration}"
        artifact_dir.mkdir(parents=True, exist_ok=True)
        
        # Save original
        orig_path = artifact_dir / "original.wav"
        sf.write(str(orig_path), original_audio, sr)
        
        # Save perturbed
        pert_path = artifact_dir / f"perturbed_{strategy_name}.wav"
        sf.write(str(pert_path), perturbed_audio, sr)
        
        print(f"  Artifacts saved to: {artifact_dir}")


# Initialize main orchestrator
orchestrator = AgentOrchestrator(
    normalizer=normalizer,
    asr_baseline=asr_baseline,
    codec_detector=codec_detector,
    codec_stack=codec_stack,
    llm_orchestrator=llm_orchestrator,
    perturbation_executor=perturbation_executor,
    metrics_computer=metrics_computer
)

print("Main orchestrator initialized!")


## Run Experiment

Load a sample audio file and run the full pipeline.


In [ ]:
# Example: Run experiment on a sample audio file
# Replace with your audio file path
sample_audio = Path("/Users/kunal/Downloads/adversarial_dataset-A/Adversarial-Examples/short-signals/Original-examples/sample-000303.wav")

if sample_audio.exists():
    # Run experiment
    results = orchestrator.run_experiment(
        sample_audio,
        reference_transcript=None,  # Will be auto-generated
        max_iterations=MAX_ITERATIONS
    )
    
    # Save results
    results_path = RESULTS_DIR / f"experiment_{sample_audio.stem}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    orchestrator.save_results(results, results_path)
    
    print(f"\n{'='*80}")
    print("Experiment completed!")
    print(f"{'='*80}\n")
else:
    print(f"Audio file not found: {sample_audio}")
    print("Please update the sample_audio path above.")


## Step 7: Ablations and Statistical Analysis


In [ ]:
def perform_ablations(results: List[ExperimentResult]) -> Dict[str, Any]:
    """Perform ablation studies on results."""
    ablations = {}
    
    # Convert to DataFrame for easier analysis
    data = []
    for result in results:
        data.append({
            "iteration": result.iteration,
            "strategy": result.strategy_name,
            "wer_delta": result.delta_metrics["wer_delta"],
            "cer_delta": result.delta_metrics["cer_delta"],
            "pesq": result.perturbed_metrics["pesq"],
            "stoi": result.perturbed_metrics["stoi"],
            "snr": result.perturbed_metrics["snr"],
            "codec": result.codec_info["codec_name"]
        })
    
    df = pd.DataFrame(data)
    
    # Ablation 1: With/without EoT
    # (Compare direct perturbation vs EoT-averaged results)
    if len(results) > 0:
        direct_wer = df["wer_delta"].mean()
        eot_wer = np.mean([
            np.mean([eot["wer_delta"] for eot in r.eot_results])
            for r in results if r.eot_results
        ])
        ablations["eot_impact"] = {
            "direct_wer_delta": direct_wer,
            "eot_wer_delta": eot_wer,
            "improvement": eot_wer - direct_wer
        }
    
    # Ablation 2: Codec-specific performance
    codec_performance = df.groupby("codec").agg({
        "wer_delta": ["mean", "std"],
        "pesq": ["mean", "std"],
        "stoi": ["mean", "std"]
    }).to_dict()
    ablations["codec_performance"] = codec_performance
    
    # Ablation 3: Strategy family comparison
    strategy_families = {}
    for result in results:
        family = result.strategy.get("family", "unknown")
        if family not in strategy_families:
            strategy_families[family] = []
        strategy_families[family].append(result.delta_metrics["wer_delta"])
    
    ablations["strategy_families"] = {
        family: {
            "mean": np.mean(deltas),
            "std": np.std(deltas),
            "count": len(deltas)
        }
        for family, deltas in strategy_families.items()
    }
    
    # Statistical confidence intervals (bootstrap)
    if len(df) > 0:
        def bootstrap_mean(data, n_bootstrap=1000, confidence=0.95):
            means = []
            for _ in range(n_bootstrap):
                sample = np.random.choice(data, size=len(data), replace=True)
                means.append(np.mean(sample))
            means = np.array(means)
            lower = np.percentile(means, (1 - confidence) / 2 * 100)
            upper = np.percentile(means, (1 - confidence) / 2 * 100 + confidence * 100)
            return {
                "mean": np.mean(means),
                "ci_lower": lower,
                "ci_upper": upper,
                "confidence": confidence
            }
        
        ablations["wer_delta_ci"] = bootstrap_mean(df["wer_delta"].values)
        ablations["cer_delta_ci"] = bootstrap_mean(df["cer_delta"].values)
    
    return ablations


def visualize_results(results: List[ExperimentResult], output_dir: Path):
    """Create visualizations of results."""
    if not results:
        print("No results to visualize")
        return
    
    # Prepare data
    data = []
    for result in results:
        data.append({
            "iteration": result.iteration,
            "strategy": result.strategy_name,
            "wer_delta": result.delta_metrics["wer_delta"],
            "cer_delta": result.delta_metrics["cer_delta"],
            "pesq": result.perturbed_metrics["pesq"],
            "stoi": result.perturbed_metrics["stoi"],
            "snr": result.perturbed_metrics["snr"]
        })
    
    df = pd.DataFrame(data)
    
    # Create plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: WER/CER delta over iterations
    ax = axes[0, 0]
    ax.plot(df["iteration"], df["wer_delta"], marker='o', label='WER Δ', linewidth=2)
    ax.plot(df["iteration"], df["cer_delta"], marker='s', label='CER Δ', linewidth=2)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Error Rate Delta")
    ax.set_title("ASR Performance Degradation Over Iterations")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Perceptual quality metrics
    ax = axes[0, 1]
    ax.plot(df["iteration"], df["pesq"], marker='o', label='PESQ', linewidth=2, color='green')
    ax2 = ax.twinx()
    ax2.plot(df["iteration"], df["stoi"], marker='s', label='STOI', linewidth=2, color='orange')
    ax.set_xlabel("Iteration")
    ax.set_ylabel("PESQ", color='green')
    ax2.set_ylabel("STOI", color='orange')
    ax.set_title("Perceptual Quality Metrics")
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Plot 3: SNR over iterations
    ax = axes[1, 0]
    ax.plot(df["iteration"], df["snr"], marker='o', linewidth=2, color='red')
    ax.axhline(y=MIN_SNR, color='r', linestyle='--', label=f'Min SNR ({MIN_SNR} dB)')
    ax.set_xlabel("Iteration")
    ax.set_ylabel("SNR (dB)")
    ax.set_title("Signal-to-Noise Ratio")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Strategy comparison
    ax = axes[1, 1]
    strategy_wer = df.groupby("strategy")["wer_delta"].mean().sort_values(ascending=False)
    ax.barh(range(len(strategy_wer)), strategy_wer.values)
    ax.set_yticks(range(len(strategy_wer)))
    ax.set_yticklabels(strategy_wer.index)
    ax.set_xlabel("Mean WER Delta")
    ax.set_title("Strategy Performance Comparison")
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plot_path = output_dir / "results_visualization.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Visualization saved to: {plot_path}")


# Run ablations and visualization if results exist
if 'results' in locals() and results:
    print("\n" + "="*80)
    print("Performing Ablations and Statistical Analysis")
    print("="*80 + "\n")
    
    ablations = perform_ablations(results)
    
    print("Ablation Results:")
    print(json.dumps(ablations, indent=2, default=str))
    
    # Save ablations
    ablations_path = RESULTS_DIR / f"ablations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(ablations_path, 'w') as f:
        json.dump(ablations, f, indent=2, default=str)
    print(f"\nAblations saved to: {ablations_path}")
    
    # Create visualizations
    visualize_results(results, RESULTS_DIR)
else:
    print("Run the experiment first to generate results for analysis.")
